<a href="https://colab.research.google.com/github/denmj/kaggle/blob/master/WiDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:

import sys

# Working with data 
import pandas as pd

# Vizualization 
import matplotlib

# math and stat lib
import numpy as np
import scipy as sp

import IPython
from IPython import display
import sklearn 
import random
import time

import warnings
warnings.filterwarnings('ignore')

In [94]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier

#Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [95]:
pd.set_option('display.max_rows', 500)


In [96]:
data_dict = pd.read_csv('/content/DataDictionaryWiDS2021.csv')

train_set = pd.read_csv('/content/TrainingWiDS2021.csv')
test_set = pd.read_csv('/content/UnlabeledWiDS2021.csv')

In [97]:
train_set.info(), test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130157 entries, 0 to 130156
Columns: 181 entries, Unnamed: 0 to diabetes_mellitus
dtypes: float64(157), int64(18), object(6)
memory usage: 179.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10234 entries, 0 to 10233
Columns: 180 entries, Unnamed: 0 to solid_tumor_with_metastasis
dtypes: float64(156), int64(18), object(6)
memory usage: 14.1+ MB


(None, None)

In [98]:
train_set.describe()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
count,130157.000000,130157.000000,130157.000000,125169.000000,125667.000000,130157.000000,128080.000000,130157.000000,130157.000000,130157.0,126694.000000,51994.000000,128472.000000,129292.000000,130157.000000,130157.000000,47597.000000,104746.000000,105275.000000,30437.000000,127967.000000,127967.000000,129448.000000,127967.000000,115461.000000,129848.000000,103399.000000,130157.000000,129737.000000,30437.000000,30437.000000,30437.000000,30437.000000,129349.000000,105638.00000,123546.000000,66990.000000,130157.000000,100682.000000,35089.000000,...,27367.000000,27367.000000,27201.000000,27201.000000,48944.000000,48944.000000,11690.000000,11690.000000,24428.000000,24428.000000,29336.000000,29336.000000,28376.000000,28376.000000,24171.000000,24171.000000,45696.000000,45696.000000,45350.000000,45350.000000,46147.000000,46147.000000,36818.000000,36818.000000,22491.000000,22491.000000,22308.000000,22308.000000,22712.000000,22712.000000,16760.000000,16760.000000,130157.00000,130157.000000,130157.000000,130157.000000,130157.000000,130157.000000,130157.000000,130157.000000
mean,65079.000000,213000.856519,106.102131,61.995103,29.110260,0.189840,169.607219,662.428344,0.839933,0.0,83.791104,2.886149,185.492683,565.994296,0.207111,0.027997,1.201222,25.718070,1.481629,0.595735,3.488290,5.484828,0.011441,4.030203,160.141416,99.854530,32.975817,0.156626,87.193046,42.161246,42.161246,132.061737,7.352154,25.150603,137.94526,36.420638,1800.803417,0.330432,12.187662,79.261563,...,11.204166,11.088205,33.731830,33.349796,1.577788,1.463473,3.028198,2.976982,193.943057,193.123506,4.188984,4.147028,138.167205,137.879814,13.387873,13.336485,45.341451,38.535587,7.387687,7.322903,165.003814,102.957476,287.600071,224.005403,44.552966,43.341081,7.337283,7.327771,163.035835,145.949537,247.525419,239.617358,0.00103,0.016081,0.013599,0.025669,0.007307,0.004187,0.020852,0.216285
std,37573.233831,38109.828146,63.482277,16.822880,8.262776,0.392176,10.833085,304.259843,2.485337,0.0,24.963063,0.689812,85.858208,466.510850,0.405238,0.164965,2.351994,20.690041,1.543535,0.262922,0.939831,1.271039,0.106349,1.538528,90.701327,30.759505,6.834576,0.363449,41.908109,12.267414,12.267414,84.958826,0.098423,15.024730,5.30384,0.857584,1456.551481,0.470370,6.931023,21.693320,...,2.350633,2.381224,6.804903,6.963975,0.946477,0.737639,2.898524,2.854953,92.486473,92.692759,0.760183,0.749463,5.711628,5.658036,6.933161,6.915142,14.623775,10.996254,0.085651,0.111530,107.084058,61.514357,130.319620,119.119675,14.631359,14.052015,0.105930,0.108658,112.646743,100.211935,131.440167,128.562211,0.03207,0.125786,0.115819,0.158146,0.085166,0.064574,0.142888,0.411712
min,1.000000,147000.000000,1.000000,0.000000,14.844926,0.000000,137.200000,82.000000,-0

In [99]:
train_set.shape, test_set.shape

# copy of orig data set 
train_set_copy = train_set.copy()
test_set_copy = test_set.copy()

In [100]:
# Cols  that are useless (idex and id cols) 
train_set_copy = train_set_copy.drop(columns=['Unnamed: 0', 'encounter_id','hospital_id','icu_id'])
test_set_copy = test_set_copy.drop(columns=['Unnamed: 0', 'encounter_id','hospital_id','icu_id'])

In [101]:
both_sets = [train_set_copy, test_set_copy]

In [103]:
cols = train_set_copy.columns
list(cols)

# 177 features (prob too many)
len(list(cols)) 

177

In [106]:
# Analysis of missing vals in data set
MVT_cols = ['Feature','Not missing vals','Total missing vals', '% of missing vals']
Missing_value_table = pd.DataFrame(columns = MVT_cols)
Missing_value_table

,Feature,Not missing vals,Total missing vals,% of missing vals


In [110]:
total_= train_set_copy.count()
missing_vals_ = train_set_copy.isnull().sum()
total_entries = train_set_copy.count().max()


In [111]:
# quick vizual on missing / na values in data set 
_index = 0
for missing, total, col_name in zip(missing_vals_, total_, cols):
  Missing_value_table.loc[_index, 'Feature'] = col_name
  Missing_value_table.loc[_index, 'Not missing vals'] = total
  Missing_value_table.loc[_index, 'Total missing vals'] = missing
  Missing_value_table.loc[_index, '% of missing vals'] = (missing/total_entries)*100
  _index += 1

In [112]:
Missing_value_table.sort_values(by='% of missing vals', ascending=False)

,Feature,Not missing vals,Total missing vals,% of missing vals
125,h1_bilirubin_max,10296,119861,92.0896
126,h1_bilirubin_min,10296,119861,92.0896
123,h1_albumin_max,11152,119005,91.4319
124,h1_albumin_min,11152,119005,91.4319
143,h1_lactate_max,11690,118467,91.0185
144,h1_lactate_min,11690,118467,91.0185
167,h1_pao2fio2ratio_max,16760,113397,87.1232
168,h1_pao2fio2ratio_min,16760,113397,87.1232
164,h1_arterial_ph_min,22308,107849,82.8607
163,h1_arterial_ph_max,22308,107849,82.8607


In [122]:
# Rule of thumb, anything with at least 50% and more missing data, candidate cols to be dropped

cols_to_drop =  list(Missing_value_table['Feature'][Missing_value_table['% of missing vals'] > 50])

In [124]:
# 73 cols to drop 
len(cols_to_drop)

73

In [126]:
# 104 to check if we can impute some data 
len(list(cols))  - len(cols_to_drop) 

104

In [128]:
# Dropping from train and test sets 
for data_set in both_sets:
    data_set.drop(cols_to_drop, axis = 1, inplace=True)

In [133]:
len(list(train_set_copy.columns)), len(list(test_set_copy.columns))

(104, 103)

In [134]:
# 1st way (Lazy way), we will fill missing cells with 0 for int/float data type 
# and with " " for categorical data type

